In [2]:
%pip install qdrant-client

  Using cached qdrant_client-1.12.0-py3-none-any.whl.metadata (10 kB)
  Using cached portalocker-2.10.1-py3-none-any.whl.metadata (8.5 kB)
  Using cached protobuf-5.28.3-cp38-abi3-macosx_10_9_universal2.whl.metadata (592 bytes)
  Using cached h2-4.1.0-py3-none-any.whl.metadata (3.6 kB)
  Using cached hyperframe-6.0.1-py3-none-any.whl.metadata (2.7 kB)
  Using cached hpack-4.0.0-py3-none-any.whl.metadata (2.5 kB)
Using cached qdrant_client-1.12.0-py3-none-any.whl (266 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.0/11.0 MB 17.4 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 12.9 MB/s eta 0:00:00a 0:00:01
Using cached portalocker-2.10.1-py3-none-any.whl (18 kB)
Using cached h2-4.1.0-py3-none-any.whl (57 kB)
Using cached hpack-4.0.0-py3-none-any.whl (32 kB)
Using cached hyperframe-6.0.1-py3-none-any.whl (12 kB)
Note: you may need to restart the kernel to use updated packages.


In [14]:
from qdrant_client import QdrantClient

client = QdrantClient(host="localhost", port=6333)

In [83]:
from qdrant_client.models import VectorParams, Distance

vector_data = [
    {"id": 1, "vector": [0.1, 0.2]},
    {"id": 2, "vector": [0.2, 0.3]},
    {"id": 3, "vector": [0.3, 0.4]},
    {"id": 4, "vector": [0.4, 0.5]},
    {"id": 5, "vector": [0.5, 0.6]},
]

if not client.collection_exists("test_collection"):
    client.create_collection(
        collection_name="test_collection",
        vectors_config=VectorParams(size=2, distance=Distance.COSINE),
    )

In [84]:
import numpy as np
from qdrant_client.models import PointStruct

vectors = np.random.rand(5, 2)
for idx, vector in enumerate(vectors):
    client.upsert(
        collection_name="test_collection",
        points=[
            PointStruct(
                id=idx,
                vector=vector.tolist(),
                payload={"color": "red"},
            )
        ],
    )

In [54]:
vectors = np.random.rand(5, 2)

# vectors[0].tolist()

print(vectors[0].tolist())

[0.3551686584952938, 0.7424673203586779]


In [85]:
client.get_collection(collection_name="test_collection")


CollectionInfo(status=<CollectionStatus.GREEN: 'green'>, optimizer_status=<OptimizersStatusOneOf.OK: 'ok'>, vectors_count=None, indexed_vectors_count=0, points_count=100, segments_count=8, config=CollectionConfig(params=CollectionParams(vectors=VectorParams(size=2, distance=<Distance.COSINE: 'Cosine'>, hnsw_config=None, quantization_config=None, on_disk=None, datatype=None, multivector_config=None), shard_number=1, sharding_method=None, replication_factor=1, write_consistency_factor=1, read_fan_out_factor=None, on_disk_payload=True, sparse_vectors=None), hnsw_config=HnswConfig(m=16, ef_construct=100, full_scan_threshold=10000, max_indexing_threads=0, on_disk=False, payload_m=None), optimizer_config=OptimizersConfig(deleted_threshold=0.2, vacuum_min_vector_number=1000, default_segment_number=0, max_segment_size=None, memmap_threshold=None, indexing_threshold=20000, flush_interval_sec=5, max_optimization_threads=None), wal_config=WalConfig(wal_capacity_mb=32, wal_segments_ahead=0), quant

In [86]:
client.get_collection(collection_name="test_collection")

CollectionInfo(status=<CollectionStatus.GREEN: 'green'>, optimizer_status=<OptimizersStatusOneOf.OK: 'ok'>, vectors_count=None, indexed_vectors_count=0, points_count=100, segments_count=8, config=CollectionConfig(params=CollectionParams(vectors=VectorParams(size=2, distance=<Distance.COSINE: 'Cosine'>, hnsw_config=None, quantization_config=None, on_disk=None, datatype=None, multivector_config=None), shard_number=1, sharding_method=None, replication_factor=1, write_consistency_factor=1, read_fan_out_factor=None, on_disk_payload=True, sparse_vectors=None), hnsw_config=HnswConfig(m=16, ef_construct=100, full_scan_threshold=10000, max_indexing_threads=0, on_disk=False, payload_m=None), optimizer_config=OptimizersConfig(deleted_threshold=0.2, vacuum_min_vector_number=1000, default_segment_number=0, max_segment_size=None, memmap_threshold=None, indexing_threshold=20000, flush_interval_sec=5, max_optimization_threads=None), wal_config=WalConfig(wal_capacity_mb=32, wal_segments_ahead=0), quant

In [87]:
client.scroll(collection_name="test_collection", limit=5, with_vectors=True)

([Record(id=0, payload={'color': 'red'}, vector=[0.6077387, 0.79413706], shard_key=None, order_value=None),
  Record(id=1, payload={'color': 'red'}, vector=[0.5660218, 0.82439023], shard_key=None, order_value=None),
  Record(id=2, payload={'color': 'red'}, vector=[0.66322464, 0.74842036], shard_key=None, order_value=None),
  Record(id=3, payload={'color': 'red'}, vector=[0.26596117, 0.9639838], shard_key=None, order_value=None),
  Record(id=4, payload={'color': 'red'}, vector=[0.3843717, 0.9231784], shard_key=None, order_value=None)],
 5)

In [92]:
client.query_points(
    collection_name="test_collection",
    query=vectors[0].tolist(),
    with_vectors=True,
)

QueryResponse(points=[ScoredPoint(id=0, version=18, score=1.0, payload={'color': 'red'}, vector=[0.6077387, 0.79413706], shard_key=None, order_value=None), ScoredPoint(id=82, version=0, score=0.9999949, payload={'color': 'red', 'rand_number': 2}, vector=[0.61026424, 0.7921979], shard_key=None, order_value=None), ScoredPoint(id=50, version=0, score=0.99997854, payload={'color': 'red', 'rand_number': 0}, vector=[0.61291414, 0.79014945], shard_key=None, order_value=None), ScoredPoint(id=12, version=0, score=0.99986804, payload={'color': 'red', 'rand_number': 2}, vector=[0.620561, 0.78415823], shard_key=None, order_value=None), ScoredPoint(id=97, version=0, score=0.9998661, payload={'color': 'red', 'rand_number': 7}, vector=[0.5946645, 0.8039739], shard_key=None, order_value=None), ScoredPoint(id=32, version=0, score=0.99970794, payload={'color': 'red', 'rand_number': 2}, vector=[0.62675285, 0.77921814], shard_key=None, order_value=None), ScoredPoint(id=44, version=0, score=0.9996995, payl